In [1]:
from ray.tune.schedulers import PopulationBasedTraining
from ray import tune, air
from ray.rllib.algorithms.ppo import PPO, PPOConfig
# TODO relative import
from envs import MultiAgentBuildingEnv, MonoAgentBuildingEnv

E0000 00:00:1732186961.580770   61538 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1732186961.584270   61538 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [ ]:
pbt = PopulationBasedTraining(
    time_attr="training_iteration",
    perturbation_interval=4,
    resample_probability=0.25,
    hyperparam_mutations={
        "lr": tune.uniform(1e-5, 0.1),
        "batch_mode": ["complete_episodes"],
        "train_batch_size": [4000],
        "sgd_minibatch_size": [32, 64, 128, 256, 512],
        "num_sgd_iter": [10, 20, 30],
        "clip_param": tune.uniform(0.1, 0.3),
    },
)

tuner = tune.Tuner(
    PPO,
    param_space=(
        #MultiAgentBuildingEnv.get_algo_config(PPOConfig())
        MonoAgentBuildingEnv.get_algo_config(
            PPOConfig()
            # .api_stack(enable_rl_module_and_learner=True,
            #            enable_env_runner_and_connector_v2=True)
            .rollouts(
                num_rollout_workers=2, 
                # rollout_fragment_length='auto',
                rollout_fragment_length=2000,
            )
            .resources(num_gpus=0.25)
        )
    ),
    tune_config=tune.TuneConfig(
        #reuse_actors=True,
        scheduler=pbt,
        num_samples=2,
        metric="env_runners/episode_reward_mean",
        mode="max",
    ),
    run_config=air.RunConfig(
        stop={"training_iteration": 200},
        checkpoint_config=air.CheckpointConfig(
            checkpoint_at_end=True
        ),
        verbose=2,
    )
)
results = tuner.fit()

2024-11-21 11:02:48,024	WARNING deprecation.py:50 -- DeprecationWarning: `rollouts` has been deprecated. Use `AlgorithmConfig.env_runners(..)` instead. This will raise an error in the future!
2024-11-21 11:02:48,025	WARNING deprecation.py:50 -- DeprecationWarning: `AlgorithmConfig.env_runners(num_rollout_workers)` has been deprecated. Use `AlgorithmConfig.env_runners(num_env_runners)` instead. This will raise an error in the future!
/home/AD/user/lab/reports/2024xxxx/.venv/lib/python3.11/site-packages/ray/_private/node.py:1362: ResourceWarning: unclosed file <_io.TextIOWrapper name='/tmp/ray/session_2024-11-21_11-02-48_043027_61538/logs/gcs_server.out' mode='a' encoding='utf-8'>
  self.start_gcs_server()
/home/AD/user/lab/reports/2024xxxx/.venv/lib/python3.11/site-packages/ray/_private/node.py:1362: ResourceWarning: unclosed file <_io.TextIOWrapper name='/tmp/ray/session_2024-11-21_11-02-48_043027_61538/logs/gcs_server.err' mode='a' encoding='utf-8'>
  self.start_gcs_server()
/home/AD/

(pid=63220) WARNING: All log messages before absl::InitializeLog() is called are written to STDERR
(pid=63220) E0000 00:00:1732186970.138689   63220 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
(pid=63220) E0000 00:00:1732186970.142112   63220 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
(pid=63403) WARNING: All log messages before absl::InitializeLog() is called are written to STDERR [repeated 2x across cluster] (Ray deduplicates logs by default. Set RAY_DEDUP_LOGS=0 to disable log deduplication, or see https://docs.ray.io/en/master/ray-observability/user-guides/configure-logging.html#log-deduplication for more options.)
(pid=63403) E0000 00:00:1732186976.862335   63403 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registe

Trial name,agent_timesteps_total,counters,custom_metrics,env_runners,episode_media,info,num_agent_steps_sampled,num_agent_steps_sampled_lifetime,num_agent_steps_trained,num_env_steps_sampled,num_env_steps_sampled_lifetime,num_env_steps_sampled_this_iter,num_env_steps_sampled_throughput_per_sec,num_env_steps_trained,num_env_steps_trained_this_iter,num_env_steps_trained_throughput_per_sec,num_healthy_workers,num_in_flight_async_sample_reqs,num_remote_worker_restarts,num_steps_trained_this_iter,perf,timers
PPO_MonoAgentBuildingEnv_255c0_00000,0,"{'num_env_steps_sampled': 0, 'num_env_steps_trained': 0, 'num_agent_steps_sampled': 0, 'num_agent_steps_trained': 0}",{},"{'episode_reward_max': 2.154386316892035, 'episode_reward_min': 1.816233508765201, 'episode_reward_mean': 1.985309912828618, 'episode_len_mean': 4608.0, 'episode_media': {}, 'episodes_timesteps_total': 9216, 'policy_reward_min': {'default_policy': 1.816233508765201}, 'policy_reward_max': {'default_policy': 2.154386316892035}, 'policy_reward_mean': {'default_policy': 1.985309912828618}, 'custom_metrics': {}, 'hist_stats': {'episode_reward': [2.154386316892035, 1.816233508765201], 'episode_lengths': [4608, 4608], 'policy_default_policy_reward': [2.154386316892035, 1.816233508765201]}, 'sampler_perf': {'mean_raw_obs_processing_ms': 0.287580871454917, 'mean_inference_ms': 2.159228030144076, 'mean_action_processing_ms': 0.3265244132159194, 'mean_env_wait_ms': 29.151690459100454, 'mean_env_render_ms': 0.0}, 'num_faulty_episodes': 0, 'connector_metrics': {}, 'num_episodes': 2, 'episode_return_max': 2.154386316892035, 'episode_return_min': 1.816233508765201, 'episode_return_mean': 1.985309912828618, 'episodes_this_iter': 2}",{},"{'learner': {}, 'num_env_steps_sampled': 0, 'num_env_steps_trained': 0, 'num_agent_steps_sampled': 0, 'num_agent_steps_trained': 0}",0,0,0,0,0,0,0,0,0,0,2,0,0,0,"{'cpu_util_percent': 27.2978021978022, 'ram_util_percent': 39.02197802197802}","{'training_iteration_time_ms': 60005.078, 'restore_workers_time_ms': 0.013, 'training_step_time_ms': 60005.039, 'sample_time_ms': 60005.031}"
PPO_MonoAgentBuildingEnv_255c0_00001,0,"{'num_env_steps_sampled': 0, 'num_env_steps_trained': 0, 'num_agent_steps_sampled': 0, 'num_agent_steps_trained': 0}",{},"{'episode_reward_max': 2.1611942429282496, 'episode_reward_min': 1.4966035420347044, 'episode_reward_mean': 1.8288988924814769, 'episode_len_mean': 4608.0, 'episode_media': {}, 'episodes_timesteps_total': 9216, 'policy_reward_min': {'default_policy': 1.4966035420347044}, 'policy_reward_max': {'default_policy': 2.1611942429282496}, 'policy_reward_mean': {'default_policy': 1.8288988924814769}, 'custom_metrics': {}, 'hist_stats': {'episode_reward': [1.4966035420347044, 2.1611942429282496], 'episode_lengths': [4608, 4608], 'policy_default_policy_reward': [1.4966035420347044, 2.1611942429282496]}, 'sampler_perf': {'mean_raw_obs_processing_ms': 0.2879729591898424, 'mean_inference_ms': 2.11589863124747, 'mean_action_processing_ms': 0.32197108311638833, 'mean_env_wait_ms': 29.244106437952585, 'mean_env_render_ms': 0.0}, 'num_faulty_episodes': 0, 'connector_metrics': {}, 'num_episodes': 2, 'episode_return_max': 2.1611942429282496, 'episode_return_min': 1.4966035420347044, 'episode_return_mean': 1.8288988924814769, 'episodes_this_iter': 2}",{},"{'learner': {}, 'num_env_steps_sampled': 0, 'num_env_steps_trained': 0, 'num_agent_steps_sampled': 0, 'num_agent_steps_trained': 0}",0,0,0,0,0,0,0,0,0,0,2,0,0,0,"{'cpu_util_percent': 27.301111111111116, 'ram_util_percent': 39.025555555555556}","{'training_iteration_time_ms': 60005.178, 'restore_workers_time_ms': 0.014, 'training_step_time_ms': 60005.131, 'sample_time_ms': 60005.122}"


(PPO pid=63220) 2024-11-21 11:05:11,429	WARNING rollout_ops.py:115 -- No samples returned from remote workers. If you have a slow environment or model, consider increasing the `sample_timeout_s` or decreasing the `rollout_fragment_length` in `AlgorithmConfig.env_runners(). [repeated 2x across cluster]
(PPO pid=63220) 2024-11-21 11:06:13,475	WARNING rollout_ops.py:115 -- No samples returned from remote workers. If you have a slow environment or model, consider increasing the `sample_timeout_s` or decreasing the `rollout_fragment_length` in `AlgorithmConfig.env_runners(). [repeated 2x across cluster]


In [ ]:
best_result = results.get_best_result()
print("Best Hyperparameters found: ", best_result)

Best Hyperparameters found:  Result(
  metrics={'custom_metrics': {}, 'episode_media': {}, 'info': {'learner': {'default_policy': {'custom_metrics': {}, 'learner_stats': {'cur_kl_coeff': 0.10000000000000002, 'cur_lr': 5.0000000000000016e-05, 'total_loss': 0.8455792117858887, 'policy_loss': -0.004752510281339768, 'vf_loss': 0.8493343554815197, 'vf_explained_var': 0.0013421098391215005, 'kl': 0.009973664808097126, 'entropy': 5.589063791049424, 'entropy_coeff': 0.0}, 'model': {}, 'num_grad_updates_lifetime': 109725.5, 'diff_num_grad_updates_vs_sampler_policy': 464.5}}, 'num_env_steps_sampled': 632000, 'num_env_steps_trained': 632000, 'num_agent_steps_sampled': 632000, 'num_agent_steps_trained': 632000}, 'env_runners': {'episode_reward_max': 1.1336566971035222, 'episode_reward_min': -1.1976535626154303, 'episode_reward_mean': -0.006979564999740355, 'episode_len_mean': 4608.0, 'episode_media': {}, 'episodes_timesteps_total': 460800, 'policy_reward_min': {'default_policy': -1.197653562615430

In [1]:
from ray.rllib.algorithms.ppo import PPO, PPOConfig
from ray.rllib.algorithms.callbacks import DefaultCallbacks

from controllables.core import BaseVariable, ComputedVariable
from controllables.core.tools.records import VariableRecords

import pythermalcomfort as pytc
import numpy as _numpy_


from controllables.energyplus import Actuator, OutputVariable, OutputMeter

# TODO relative import
from envs import MultiAgentBuildingEnv, MonoAgentBuildingEnv


class PMVVariable(BaseVariable):
    def __init__(
        self, 
        tdb: BaseVariable,
        tr: BaseVariable,
        rh: BaseVariable,
        metab_rate=1.5, clothing=.5, pmv_limit=.5,
    ):
        self.tdb = tdb
        self.tr = tr
        self.rh = rh
        self._metab_rate = _numpy_.asarray(metab_rate)
        self._clothing = _numpy_.asarray(clothing)
        self._pmv_limit = _numpy_.asarray(pmv_limit)
    
    @property
    def value(self):
        res = pytc.models.pmv_ppd(
            tdb=self.tdb.value, 
            tr=self.tr.value, 
            # calculate relative air speed
            vr=pytc.utilities.v_relative(v=0.1, met=self._metab_rate), 
            rh=self.rh.value, 
            met=self._metab_rate, 
            # calculate dynamic clothing
            clo=pytc.utilities.clo_dynamic(clo=self._clothing, met=self._metab_rate),
            limit_inputs=False,
        )['pmv']

        return res

class PlottingCallbacks(DefaultCallbacks):
    def __init__(self):
        self.env_records: dict[object, VariableRecords] = None

    def on_episode_start(self, *, episode, worker, **kwargs):
        env: MonoAgentBuildingEnv = worker.env
        system = env.system
        if self.env_records is None:
            system.add('logging:progress')
            self.env_records = dict()

            for key in env.agent.observation_space:
                if key == 'CHILLER' or key == 'AHU':
                    # self.env_records[agent_ref] = records = VariableRecords({
                    # 'AHU COOLING COIL': system[OutputVariable.Ref('Cooling Coil Total Cooling Rate', 'AIR LOOP AHU COOLING COIL')],
                    # 'Electricity': system[OutputMeter.Ref('Electricity:HVAC')]
                    # }, maxlen=10_000)
                    pass
                else:
                    tdb = env.agent.observation[key]['temperature:drybulb']
                    tr = env.agent.observation[key]['temperature:radiant']
                    rh = env.agent.observation[key]['humidity']
                    pmv = PMVVariable(tdb=tdb, tr=tr, rh=rh)
                    self.env_records[key] = records = VariableRecords({
                        '🕰️': system['time'],
                        '🍩': env.agent.reward,
                        'pmv': pmv,
                        'occupancy': env.agent.observation[key]['occupancy'],
                        # 'tstat': env.agents[agent_ref].action['thermostat'],
                        'temp': tdb,
                        # 'AHU COOLING COIL': system[OutputVariable.Ref('Cooling Coil Total Cooling Rate', 'AIR LOOP AHU COOLING COIL')],
                        'elec': system[OutputMeter.Ref('Electricity:HVAC')],
                    })
                    # display(
                    #     records.plot.scatter(x='🕰️', y='pmv', label=repr(key))
                    #     .watch(records.events['change'] % 1_000)
                    # )

    def on_episode_step(self, *, episode, **kwargs):
        for _, env_records in self.env_records.items():
            env_records.poll()

    def on_episode_end(self, *, episode, **kwargs):
        for agent_ref, env_records in self.env_records.items():
            env_records.dataframe().to_csv(f'backup/datasave/20241121/records_single_agent_angle_with_distance/{agent_ref}.csv', index=False)


E0000 00:00:1732177809.122615   17169 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1732177809.125936   17169 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [2]:
config_eval = (
    MonoAgentBuildingEnv.get_algo_config(
        PPOConfig()
        .rollouts(
            num_rollout_workers=4, 
            rollout_fragment_length='auto',
        )
        .resources(num_gpus=1/2)
    )
    .env_runners(
        num_env_runners=0,
        create_env_on_local_worker=True,
    )
    .evaluation(
        evaluation_duration=1,
        evaluation_duration_unit='episodes',
        #evaluation_interval=1,
        evaluation_num_env_runners=0,
    )
    .callbacks(PlottingCallbacks)
)

algo_eval = PPO(config_eval)
#algo_eval.restore(best_result.checkpoint)
algo_eval.restore('/home/AD/user/ray_results/PPO_2024-11-21_02-45-33/PPO_MonoAgentBuildingEnv_aeca7_00001_1_2024-11-21_02-45-35/checkpoint_000045')
#algo_eval.restore("/home/AD/user/ray_results/PPO_2024-10-18_15-32-34/PPO_UserMultiAgentEnv_331fb_00000_0_2024-10-18_15-32-35/checkpoint_000013")

2024-11-21 08:30:14,792	WARNING deprecation.py:50 -- DeprecationWarning: `rollouts` has been deprecated. Use `AlgorithmConfig.env_runners(..)` instead. This will raise an error in the future!
2024-11-21 08:30:14,793	WARNING deprecation.py:50 -- DeprecationWarning: `AlgorithmConfig.env_runners(num_rollout_workers)` has been deprecated. Use `AlgorithmConfig.env_runners(num_env_runners)` instead. This will raise an error in the future!
/home/AD/user/lab/reports/2024xxxx/.venv/lib/python3.11/site-packages/ray/rllib/algorithms/algorithm.py:555: RayDeprecationWarning: This API is deprecated and may be removed in future Ray releases. You could suppress this warning by setting env variable PYTHONWARNINGS="ignore::DeprecationWarning"
`UnifiedLogger` will be removed in Ray 2.7.
  return UnifiedLogger(config, logdir, loggers=None)
/home/AD/user/lab/reports/2024xxxx/.venv/lib/python3.11/site-packages/ray/tune/logger/unified.py:53: RayDeprecationWarning: This API is deprecated and may be removed in

In [3]:
for _ in range(2):
    algo_eval.evaluate()

/home/AD/user/lab/reports/2024xxxx/.venv/lib/python3.11/site-packages/controllables/energyplus/variables.py:370: RuntimeWarning: OutputVariable(OutputVariable.Ref(type='Zone Mean Air Temperature', key='1FFIRSTFLOOREAST:OPENOFFICE')) requested while Kernel() is running; It may not be available until the next run. More info: https://energyplus.readthedocs.io/en/latest/datatransfer.html#datatransfer.DataExchange.request_variable
  _warnings_.warn(
/home/AD/user/lab/reports/2024xxxx/.venv/lib/python3.11/site-packages/controllables/energyplus/variables.py:370: RuntimeWarning: OutputVariable(OutputVariable.Ref(type='Zone Mean Radiant Temperature', key='1FFIRSTFLOOREAST:OPENOFFICE')) requested while Kernel() is running; It may not be available until the next run. More info: https://energyplus.readthedocs.io/en/latest/datatransfer.html#datatransfer.DataExchange.request_variable
  _warnings_.warn(
/home/AD/user/lab/reports/2024xxxx/.venv/lib/python3.11/site-packages/controllables/energyplus/var

  0%|          | 0/100 [00:00<?, ?it/s]